In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
os.getcwd()

'/Users/caseyh/Desktop/footyfootyfooty/data_wrangling'

In [4]:
path = '/Users/caseyh/Desktop/footyfootyfooty/stg_tables/'
teams_str = ['adelaide', 'brisbanel','carlton', 'collingwood','essendon', 'fremantle','geelong','goldcoast','gws',
        'hawthorn', 'melbourne', 'kangaroos','padelaide','richmond','stkilda','swans','westcoast','bullldogs']

In [5]:
stat_str = ['Kicks.csv',
 'Frees Against.csv',
 'Tackles.csv',
 'Bounces.csv',
 'Marks.csv',
 'Behinds.csv',
 'Hit Outs.csv',
 'Contested Marks.csv',
 'Disposals.csv',
 'Clearances.csv',
 'Inside 50s.csv',
 'Handballs.csv',
 'Contested Possessions.csv',
 'Clangers.csv',
 'Subs.csv',
 'Uncontested Possessions.csv',
 'Frees.csv',
 'Marks Inside 50.csv',
 'Goal Assists.csv',
 'Rebounds.csv',
 '% Played.csv',
 'Goals.csv',
 'One Percenters.csv',
'Brownlow Votes.csv']
cols = ['Player', 'year', 'round', 'team', 'opponent', 'Disposals','Kicks', 'Handballs','Goals','Behinds', 'Marks',
        'Clearances','Tackles','Contested Possessions','Uncontested Possessions',\
        'Hit Outs', 'Contested Marks','Marks Inside 50','Goal Assists',\
        'Frees','Frees Against',  \
         'Inside 50s','Rebounds', '% Played', 'Subs',\
       'Clangers','One Percenters', 'Bounces', 'Brownlow Votes']
num_cols = ['Disposals','Kicks', 'Handballs','Frees Against', 'Tackles', 'Bounces',\
       'Marks', 'Behinds', 'Hit Outs', 'Contested Marks',\
       'Clearances', 'Inside 50s', 'Contested Possessions',\
       'Clangers', 'Uncontested Possessions', 'Frees',\
       'Marks Inside 50', 'Goal Assists', 'Rebounds', 'Goals',\
       'One Percenters', 'Brownlow Votes']

In [6]:
# Data clean
# -, nan removal
# convert str to int
def clean(value):
    if value == '-':
        return 0
    elif type(value) == float:
        return np.nan
    elif type(int(value)) == int:
        return int(value)

In [7]:
# initial stat table 
for year in range(2023,2024):
    for team in teams_str:
        output = pd.DataFrame({}, columns=cols)
        try:
            table = pd.read_csv(f'{path}{team}_{year}/{stat_str[0]}').drop(columns = 'Tot')
            
        except:
            continue
        
        stat = stat_str[0].replace('.csv','')
        stats = table[(table['Player'] != 'Totals') & (table['Player'] != 'Opponent')]
        tmp = pd.melt(stats, id_vars=['Player']).rename(columns={'variable':'round','value':stat})

        # join rest of tables
        for s in stat_str[1:]:
            
            stat = s.replace('.csv','')
            try:
                table = pd.read_csv(f'{path}{team}_{year}/{s}').drop(columns = 'Tot')
                stats = table[(table['Player'] != 'Totals') & (table['Player'] != 'Opponent')]
                players_f = pd.melt(stats, id_vars=['Player']).rename(columns={'variable':'round','value':stat})
                tmp = tmp.merge(players_f, how = 'left', on = ['Player', 'round'])
            except:
                continue

        # get opponents
        opponent_f = table[table['Player'] == 'Opponent'].melt(id_vars = 'Player').drop(columns='Player').rename(columns={'variable':'round','value':'opponent'})

        tmp = tmp.merge(opponent_f, on= 'round')

        tmp['team'] = team
        tmp['year'] = year
        
        tmp = tmp.fillna(np.nan)
        
        
        
        output = pd.concat([output,tmp], axis=0, ignore_index=True)
        for col in num_cols:
            try:
                output[col] = output[col].apply(lambda x: clean(x))
            except:
                continue
        print(year, team)

        output.to_csv(f'/Users/caseyh/Desktop/footyfootyfooty/playerstats/{team}_{year}_player_stats.csv', index = False)


2023 adelaide
2023 brisbanel
2023 carlton
2023 collingwood
2023 essendon
2023 fremantle
2023 geelong
2023 goldcoast
2023 gws
2023 hawthorn
2023 melbourne
2023 kangaroos
2023 padelaide
2023 richmond
2023 stkilda
2023 swans
2023 westcoast
2023 bullldogs


## All Player Stats Combined

In [8]:
path = '/Users/caseyh/Desktop/footyfootyfooty/playerstats/'

In [9]:
opponent_mapping = {'RI':'RCH','GE':'GEE','NM':'NTH','PA':'POR','GC':'GCS','ME':'MEL','GW':'GWS','HW':'HAW','CA':'CAR','BL':'BRL','CW':'COL','AD':'ADE','FR':'FRE','WB':'WBD','FO':'WBD','SY':'SYD','WC':'WCE','SK':'STK','ES':'ESS'}
team_mapping = {'richmond':'RCH','geelong':'GEE','kangaroos':'NTH','padelaide':'POR','goldcoast':'GCS','melbourne':'MEL','gws':'GWS','hawthorn':'HAW','carlton':'CAR','brisbanel':'BRL','collingwood':'COL','adelaide':'ADE','fremantle':'FRE','bullldogs':'WBD','swans':'SYD','westcoast':'WCE','stkilda':'STK','essendon':'ESS'}

In [10]:
def rearrange(value):
    return ' '.join(' '.join(value.lower().split(', ')[::-1]).split('-'))

In [10]:
for year in range(2023,2024):
    output = pd.DataFrame({})
    
    for team in teams_str:
        try:
            tmp = pd.read_csv(f'/Users/caseyh/Desktop/footy/playerstats/{team}_{year}_player_stats.csv')
        except:
            continue
            
        output = pd.concat([output,tmp])
    
    for key in opponent_mapping.keys():
        output.loc[output['opponent']==key, 'opponent'] = opponent_mapping[key]
    for key in team_mapping.keys():
        output.loc[output['team']==key, 'team'] = team_mapping[key]
    output['Player'] = output['Player'].apply(lambda x: rearrange(x))
    
    output.to_csv(f'/Users/caseyh/Desktop/footy/seasonstats/{year}_player_stats.csv', index = False)